In [1]:
import seaborn as sns
import pandas as pd
import pickle

sns.set_style("darkgrid")
pd.set_option('display.max_columns', 999)
pd.set_option('display.max_rows', 999)

######################################################################

import requests
from bs4 import BeautifulSoup

######################################################################

from src.data_cleaning import *

######################################################################

import warnings
warnings.filterwarnings('ignore')

# Data Understanding

### This dataset comes from two sources being: 

* NHL API - player stats by season
* [Spotrac.com Webscraping](www.sportrac.com) - player salaries by season

### Below we will start our data collections. 

### To start, the below cell pulls from the NHL API and extracts all NHL team information and stores the requests objects in a dictionary. We then remove all values that return a 404 error and, store the object in a pickle file, and, finally, convert the raw data into a dataframe. 

#### Note: All pickled objects can be found in the json_files folder

#### Note: This process is identifcal for all NHL API pulls. The NHL API pulls end at cell 24.

In [2]:
# nhl_teams = {}

# for team in range(1,102):
    
#     url = f"https://statsapi.web.nhl.com/api/v1/teams/{team}"

#     headers = {
#         'x-rapidapi-key': "c10092ada9mshd8d43cac6b3023cp1b369ajsn4c45f8f530d0",
#         'x-rapidapi-host': "api-hockey.p.rapidapi.com"
#     }

#     nhl_teams[team] = requests.request("GET", url, headers=headers)


In [3]:
# errors_404 = [59, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 83, 84, 85, 86, 99]

# for i in errors_404:
#     nhl_teams.pop(i)

In [4]:
# pickle_out = open("json_files/nhl_teams.pickle","wb")
# pickle.dump(nhl_teams, pickle_out)
# pickle_out.close()

pickle_in = open("json_files/nhl_teams.pickle","rb")
nhl_teams = pickle.load(pickle_in)

all_teams_df = get_teams(nhl_teams)
all_teams_df.head()

,id,name,link,venue_name,venue_link,venue_city,venue_timeZone,abbreviation,teamName,locationName,firstYearOfPlay,division_id,division_name,division_link,conference_id,conference_name,conference_link,franchise_franchiseId,franchise_teamName,franchise_link,shortName,officialSiteUrl,franchiseId,active,venue_id,division_nameShort,division_abbreviation
1,1,New Jersey Devils,/api/v1/teams/1,Prudential Center,/api/v1/venues/null,Newark,America/New_York,NJD,Devils,New Jersey,1982,25,MassMutual East,/api/v1/divisions/25,6,Eastern,/api/v1/conferences/6,23,Devils,/api/v1/franchises/23,New Jersey,http://www.newjerseydevils.com/,23,True,NaN,NaN,NaN
2,2,New York Islanders,/api/v1/teams/2,Nassau Veterans Memorial Coliseum,/api/v1/venues/null,Uniondale,America/New_York,NYI,Islanders,New York,1972,25,MassMutual East,/api/v1/divisions/25,6,Eastern,/api/v1/conferences/6,22,Islanders,/api/v1/franchises/22,NY Islanders,http://www.newyorkislanders.com/,22,True,NaN,NaN,NaN
3,3,New York Rangers,/api/v1/teams/3,Madison Square Garden,/api/v1/venues/5054,New York,America/New_York,NYR,Rangers,New York,1926,25,MassMutual East,/api/v1/divisions/25,6,Eastern,/api/v1/conferences/6,10,Rangers,/api/v1/franchises/10,NY Rangers,http://www.newyorkrangers.com/,10,True,5054,NaN,NaN
4,4,Philadelphia Flyers,/api/v1/teams/4,Wells Fargo Center,/api/v1/venues/5096,Philadelphia,America/New_York,PHI,Flyers,Philadelphia,1967,25,MassMutual East,/api/v1/divisions/25,6,Eastern,/api/v1/conferences/6,16,Flyers,/api/v1/franchises/16,Philadelphia,http://www.philadelphiaflyers.com/,16,True,5096,NaN,NaN
5,5,Pittsburgh Penguins,/api/v1/teams/5,PPG Paints Arena,/api/v1/venues/5034,Pittsburgh,America/New_York,PIT,Penguins,Pittsburgh,1967,25,MassMutual East,/api/v1/divisions/25,6,Eastern,/api/v1/conferences/6,17,Penguins,/api/v1/franchises/17,Pittsburgh,http://pittsburghpenguins.com/,17,True,5034,NaN,NaN


#### The above dataframe contains information regarding all of the stadims in the NHL. The columns descriptions are:

* id - team ID
* name - full team name
* link - suffix of URL to access specific team's API
* venue_name - stadium name
* venue_link - suffix of URL to access specific venue's API
* venue_city - city of which the venue resides
* venue_timeZone - timezone of the venue
* abbreviation - team name abbreviation
* team_Name - shortened team name (I.E. New Jersey Devils = Devils)
* locationName - State or Province of team
* firstYearOfPlay - first year the team entered the NHL
* division_id - NHL ID number for teams division
* divion_name - name of teams NHL division
* division_link - suffix of URL to access specific division's API
* conference_id - NHL ID number for teams conference
* conference_name - name of teams conference
* conference_link - suffix of URL to access specific conference API
* franchise_franchiseId - NHL ID number for franchise
* franchise_teamName - shortened team name I.E. New Jersey Devils = Devils)
* officialSiteUrl - NHL teams website URL
* franchiseId - NHL ID number for franchise
* active - True/False on whether a team is still playing (I.E. Hartford Whalers would show false)
* venue_id - NHL ID for venues
* division_nameShort - NaN
* division_abbreviation - NaN

#### The above dataframe does not give us any player specific information but since the NHL's API is undocumented we are shooting in the dark as far as what information we will get from our requests. This information doesn't seem to helpful at the moment but it may be useful in the future.

#### Below we attempt to pull stadium and franchise information from the links associated in the above dataframe but are unable to get any return information.

In [5]:
# nhl_team_stadiums = {}

# for link in all_teams_df['venue_link']:

#     url = f"https://statsapi.web.nhl.com/api/v1/teams/{link}/roster"

#     headers = {
#         'x-rapidapi-key': "c10092ada9mshd8d43cac6b3023cp1b369ajsn4c45f8f530d0",
#         'x-rapidapi-host': "api-hockey.p.rapidapi.com"
#     }

#     nhl_team_stadiums[link] = requests.request("GET", url, headers=headers)

print('Response [404]')

Response [404]


In [6]:
# nhl_team_franchise = {}

# for link in all_teams_df['franchise_link']:

#     url = f"https://statsapi.web.nhl.com/api/v1/teams/{link}/roster"

#     headers = {
#         'x-rapidapi-key': "c10092ada9mshd8d43cac6b3023cp1b369ajsn4c45f8f530d0",
#         'x-rapidapi-host': "api-hockey.p.rapidapi.com"
#     }

#     nhl_team_franchise[link] = requests.request("GET", url, headers=headers)

print('Response [404]')

Response [404]


#### Next we move to extract team roster information from the NHL API. Again, we remove the 404 errors, store our JSON files in a dictionary, pickle the object, run our dictionary through our function, and return a pandas dataframe. 

In [7]:
# nhl_team_roster = {}

# for number in range(1,55):

#     url = f"https://statsapi.web.nhl.com/api/v1/teams/{number}/roster"

#     headers = {
#         'x-rapidapi-key': "c10092ada9mshd8d43cac6b3023cp1b369ajsn4c45f8f530d0",
#         'x-rapidapi-host': "api-hockey.p.rapidapi.com"
#     }

#     nhl_team_roster[number] = requests.request("GET", url, headers=headers)

In [8]:
# roster_error_404 = [11, 27, 31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51]
# for error in roster_error_404:
#     nhl_team_roster.pop(error)

In [9]:
# pickle_out = open("json_files/nhl_rosters.pickle","wb")
# pickle.dump(nhl_team_roster, pickle_out)
# pickle_out.close()

In [10]:
pickle_in = open("json_files/nhl_rosters.pickle","rb")
nhl_rosters = pickle.load(pickle_in)

In [11]:
rosters = build_roster(nhl_rosters)
rosters = rosters.astype('object')
rosters.head()

,id,fullName,link,jerseyNumber,code,name,type,abbreviation,Team_Number
0,8477972,Josh Jacobs,/api/v1/people/8477972,40,D,Defenseman,Defenseman,D,1
1,8478421,A.J. Greer,/api/v1/people/8478421,42,L,Left Wing,Forward,LW,1
2,8474056,P.K. Subban,/api/v1/people/8474056,76,D,Defenseman,Defenseman,D,1
3,8475809,Scott Wedgewood,/api/v1/people/8475809,41,G,Goalie,Goalie,G,1
4,8476807,Matt Tennyson,/api/v1/people/8476807,7,D,Defenseman,Defenseman,D,1


#### This dataframe looks more promising. We get a small amount of player information but we can pull more stats through the 'link' column. Below are descriptions of each column in the dataframe:

* id - NHL player id
* fullName - full name of NHL player
* link - suffix of URL to access specific players API
* jerseyNumber - team number of player
* code - abbreviation of position (I.E. D = Defenseman)
* name - full position name
* type - position type (Forward, Defensement, Goalie)
* abbreviation - full abbreviation of the name column (I.E. Left Wing = LW)
* Team_Number - NHL team number - **Note: this may tie to our teams dataframe on the id column**

#### Next we will pull player specific stats using the link column from the above dataframe & join the new dataframe with the rosters dataframe using the 'id' column

In [12]:
# player_stats = {}
# for link in rosters['link']:

#     url = f"https://statsapi.web.nhl.com/{link}"

#     headers = {
#         'x-rapidapi-key': "c10092ada9mshd8d43cac6b3023cp1b369ajsn4c45f8f530d0",
#         'x-rapidapi-host': "api-hockey.p.rapidapi.com"
#     }

#     player_stats[link] = requests.request("GET", url, headers=headers)

In [13]:
# pickle_out = open("json_files/nhl_player_stats.pickle","wb")
# pickle.dump(player_stats, pickle_out)
# pickle_out.close()

In [14]:
pickle_in = open("json_files/nhl_player_stats.pickle","rb")
nhl_player_stats = pickle.load(pickle_in)

In [15]:
player_stats = player_statistics(rosters, nhl_player_stats)
rosters = player_stats.drop_duplicates()
rosters = rosters.astype('object')
rosters.head()

,id,fullName,link,jerseyNumber,code,name,type,abbreviation,Team_Number,index,id,fullName,link,firstName,lastName,primaryNumber,birthDate,currentAge,birthCity,birthStateProvince,birthCountry,nationality,height,weight,active,alternateCaptain,captain,rookie,shootsCatches,rosterStatus,current_team_id,current_team_name,current_team_link,primary_position_code,primary_position_name,primary_position_type,primary_position_abbreviation
0,8477972,Josh Jacobs,/api/v1/people/8477972,40,D,Defenseman,Defenseman,D,1,/api/v1/people/8477972,8477972,Josh Jacobs,/api/v1/people/8477972,Josh,Jacobs,40,1996-02-15,25,Shelby Township,MI,USA,USA,"6' 2""",200,True,False,False,True,R,Y,1,New Jersey Devils,/api/v1/teams/1,D,Defenseman,Defenseman,D
1,8478421,A.J. Greer,/api/v1/people/8478421,42,L,Left Wing,Forward,LW,1,/api/v1/people/8478421,8478421,A.J. Greer,/api/v1/people/8478421,A.J.,Greer,42,1996-12-14,24,Joliette,QC,CAN,CAN,"6' 3""",210,True,False,False,False,L,Y,1,New Jersey Devils,/api/v1/teams/1,L,Left Wing,Forward,LW
2,8474056,P.K. Subban,/api/v1/people/8474056,76,D,Defenseman,Defenseman,D,1,/api/v1/people/8474056,8474056,P.K. Subban,/api/v1/people/8474056,P.K.,Subban,76,1989-05-13,32,Toronto,ON,CAN,CAN,"6' 0""",210,True,True,False,False,R,Y,1,New Jersey Devils,/api/v1/teams/1,D,Defenseman,Defenseman,D
3,8475809,Scott Wedgewood,/api/v1/people/8475809,41,G,Goalie,Goalie,G,1,/api/v1/people/8475809,8475809,Scott Wedgewood,/api/v1/people/8475809,Scott,Wedgewood,41,1992-08-14,28,Brampton,ON,CAN,CAN,"6' 2""",207,True,False,False,False,L,Y,1,New Jersey Devils,/api/v1/teams/1,G,Goalie,Goalie,G
4,8476807,Matt Tennyson,/api/v1/people/8476807,7,D,Defenseman,Defenseman,D,1,/api/v1/people/8476807,8476807,Matt Tennyson,/api/v1/people/8476807,Matt,Tennyson,7,1990-04-23,31,Minneapolis,MN,USA,USA,"6' 2""",205,True,False,False,False,R,Y,1,New Jersey Devils,/api/v1/teams/1,D,Defenseman,Defenseman,D


#### Joining the two dataframes has created redundant columns and a messy dataframe. Below we will clean up our dataframe.

#### We will start by dropping columns that we will use to merge on later on and other redundant columns that will have a negative effect on building our dataframe.

In [16]:
clean_up = rosters.iloc[:, 10:]
clean_up.drop(['fullName', 'link', 'id'], axis=1, inplace=True)
clean_up.head()

,firstName,lastName,primaryNumber,birthDate,currentAge,birthCity,birthStateProvince,birthCountry,nationality,height,weight,active,alternateCaptain,captain,rookie,shootsCatches,rosterStatus,current_team_id,current_team_name,current_team_link,primary_position_code,primary_position_name,primary_position_type,primary_position_abbreviation
0,Josh,Jacobs,40,1996-02-15,25,Shelby Township,MI,USA,USA,"6' 2""",200,True,False,False,True,R,Y,1,New Jersey Devils,/api/v1/teams/1,D,Defenseman,Defenseman,D
1,A.J.,Greer,42,1996-12-14,24,Joliette,QC,CAN,CAN,"6' 3""",210,True,False,False,False,L,Y,1,New Jersey Devils,/api/v1/teams/1,L,Left Wing,Forward,LW
2,P.K.,Subban,76,1989-05-13,32,Toronto,ON,CAN,CAN,"6' 0""",210,True,True,False,False,R,Y,1,New Jersey Devils,/api/v1/teams/1,D,Defenseman,Defenseman,D
3,Scott,Wedgewood,41,1992-08-14,28,Brampton,ON,CAN,CAN,"6' 2""",207,True,False,False,False,L,Y,1,New Jersey Devils,/api/v1/teams/1,G,Goalie,Goalie,G
4,Matt,Tennyson,7,1990-04-23,31,Minneapolis,MN,USA,USA,"6' 2""",205,True,False,False,False,R,Y,1,New Jersey Devils,/api/v1/teams/1,D,Defenseman,Defenseman,D


#### Next we will join our dataframes

In [17]:
rosters = pd.concat([rosters.iloc[:,:9], clean_up], axis=1)
rosters.head()

,id,fullName,link,jerseyNumber,code,name,type,abbreviation,Team_Number,firstName,lastName,primaryNumber,birthDate,currentAge,birthCity,birthStateProvince,birthCountry,nationality,height,weight,active,alternateCaptain,captain,rookie,shootsCatches,rosterStatus,current_team_id,current_team_name,current_team_link,primary_position_code,primary_position_name,primary_position_type,primary_position_abbreviation
0,8477972,Josh Jacobs,/api/v1/people/8477972,40,D,Defenseman,Defenseman,D,1,Josh,Jacobs,40,1996-02-15,25,Shelby Township,MI,USA,USA,"6' 2""",200,True,False,False,True,R,Y,1,New Jersey Devils,/api/v1/teams/1,D,Defenseman,Defenseman,D
1,8478421,A.J. Greer,/api/v1/people/8478421,42,L,Left Wing,Forward,LW,1,A.J.,Greer,42,1996-12-14,24,Joliette,QC,CAN,CAN,"6' 3""",210,True,False,False,False,L,Y,1,New Jersey Devils,/api/v1/teams/1,L,Left Wing,Forward,LW
2,8474056,P.K. Subban,/api/v1/people/8474056,76,D,Defenseman,Defenseman,D,1,P.K.,Subban,76,1989-05-13,32,Toronto,ON,CAN,CAN,"6' 0""",210,True,True,False,False,R,Y,1,New Jersey Devils,/api/v1/teams/1,D,Defenseman,Defenseman,D
3,8475809,Scott Wedgewood,/api/v1/people/8475809,41,G,Goalie,Goalie,G,1,Scott,Wedgewood,41,1992-08-14,28,Brampton,ON,CAN,CAN,"6' 2""",207,True,False,False,False,L,Y,1,New Jersey Devils,/api/v1/teams/1,G,Goalie,Goalie,G
4,8476807,Matt Tennyson,/api/v1/people/8476807,7,D,Defenseman,Defenseman,D,1,Matt,Tennyson,7,1990-04-23,31,Minneapolis,MN,USA,USA,"6' 2""",205,True,False,False,False,R,Y,1,New Jersey Devils,/api/v1/teams/1,D,Defenseman,Defenseman,D


#### Finally we will drop redundant columns that will be unneeded in our model building process.

In [18]:
rosters = rosters.drop(['firstName',
                         'lastName',
                         'primaryNumber',
                         'current_team_id',
                         'current_team_name',
                         'current_team_link',
                         'primary_position_code',
                         'primary_position_name',
                         'primary_position_type',
                         'primary_position_abbreviation'], axis=1)

In [19]:
rosters.head()

,id,fullName,link,jerseyNumber,code,name,type,abbreviation,Team_Number,birthDate,currentAge,birthCity,birthStateProvince,birthCountry,nationality,height,weight,active,alternateCaptain,captain,rookie,shootsCatches,rosterStatus
0,8477972,Josh Jacobs,/api/v1/people/8477972,40,D,Defenseman,Defenseman,D,1,1996-02-15,25,Shelby Township,MI,USA,USA,"6' 2""",200,True,False,False,True,R,Y
1,8478421,A.J. Greer,/api/v1/people/8478421,42,L,Left Wing,Forward,LW,1,1996-12-14,24,Joliette,QC,CAN,CAN,"6' 3""",210,True,False,False,False,L,Y
2,8474056,P.K. Subban,/api/v1/people/8474056,76,D,Defenseman,Defenseman,D,1,1989-05-13,32,Toronto,ON,CAN,CAN,"6' 0""",210,True,True,False,False,R,Y
3,8475809,Scott Wedgewood,/api/v1/people/8475809,41,G,Goalie,Goalie,G,1,1992-08-14,28,Brampton,ON,CAN,CAN,"6' 2""",207,True,False,False,False,L,Y
4,8476807,Matt Tennyson,/api/v1/people/8476807,7,D,Defenseman,Defenseman,D,1,1990-04-23,31,Minneapolis,MN,USA,USA,"6' 2""",205,True,False,False,False,R,Y


#### Below are the column descriptions for our newly created dataframe: 
**Same as from the roster API pull above**
* id - NHL player id
* fullName - full name of NHL player
* link - suffix of URL to access specific players API
* jerseyNumber - team number of player
* code - abbreviation of position (I.E. D = Defenseman)
* name - full position name
* type - position type (Forward, Defensement, Goalie)
* abbreviation - full abbreviation of the name column (I.E. Left Wing = LW)
* Team_Number - NHL team number - **Note: this may tie to our teams dataframe on the id column**

**New Columns**
* birthDate - year/month/day a player was born
* currentAge - age of player
* birthCity - city a player was born
* birthStateProvince - city/province a player was born
* birthCountry - country a player was born
* nationality - nationality of player
* height - height of player
* weight - weight of player
* active - if the player is active or not (True/False)
* alternateCaptain - if the player is an alternative captain (True/False)
* captain - if the player is a captain (True/False)
* rookie - if the player is a rookie or not (True/False)
* shootsCatches - handedness of the player 
* rosterStatus - if the player is on the roster or injury reserve


#### For our final use case with the NHL API we will pull player stats by season going back to 2013-2014 which is as far back as the API will allow. Theres no specific reason for going back this far. At this point, the more data the better.

#### We start by running our rosters through a function which pulls by year than pickle the objects to avoid multiple API pulls.

In [20]:
# player_stats_20212022 = player_stats_by_year(rosters, '20212022' )
# player_stats_20202021 = player_stats_by_year(rosters, '20202021' )
# player_stats_20192020 = player_stats_by_year(rosters, '20192020' )
# player_stats_20182019 = player_stats_by_year(rosters, '20182019' )
# player_stats_20172018 = player_stats_by_year(rosters, '20172018' )
# player_stats_20162017 = player_stats_by_year(rosters, '20162017' )
# player_stats_20152016 = player_stats_by_year(rosters, '20152016' )
# player_stats_20142015 = player_stats_by_year(rosters, '20142015' )
# player_stats_20132014 = player_stats_by_year(rosters, '20132014' )

In [21]:
# pickle_out = open("json_files/nhl_player_stats20132014.pickle","wb")
# pickle.dump(player_stats_20132014, pickle_out)
# pickle_out.close()

# pickle_out = open("json_files/nhl_player_stats20142015.pickle","wb")
# pickle.dump(player_stats_20142015, pickle_out)
# pickle_out.close()

# pickle_out = open("json_files/nhl_player_stats20152016.pickle","wb")
# pickle.dump(player_stats_20152016, pickle_out)
# pickle_out.close()

# pickle_out = open("json_files/nhl_player_stats20162017.pickle","wb")
# pickle.dump(player_stats_20162017, pickle_out)
# pickle_out.close()

# pickle_out = open("json_files/nhl_player_stats20172018.pickle","wb")
# pickle.dump(player_stats_20172018, pickle_out)
# pickle_out.close()

# pickle_out = open("json_files/nhl_player_stats20182019.pickle","wb")
# pickle.dump(player_stats_20182019, pickle_out)
# pickle_out.close()

# pickle_out = open("json_files/nhl_player_stats20192020.pickle","wb")
# pickle.dump(player_stats_20192020, pickle_out)
# pickle_out.close()

# pickle_out = open("json_files/nhl_player_stats20202021.pickle","wb")
# pickle.dump(player_stats_20202021, pickle_out)
# pickle_out.close()

# pickle_out = open("json_files/nhl_player_stats20212022.pickle","wb")
# pickle.dump(player_stats_20212022, pickle_out)
# pickle_out.close()

In [22]:
pickle_in = open("json_files/nhl_player_stats20132014.pickle","rb")
player_stats_20132014 = pickle.load(pickle_in)

pickle_in = open("json_files/nhl_player_stats20142015.pickle","rb")
player_stats_20142015 = pickle.load(pickle_in)

pickle_in = open("json_files/nhl_player_stats20152016.pickle","rb")
player_stats_20152016 = pickle.load(pickle_in)

pickle_in = open("json_files/nhl_player_stats20162017.pickle","rb")
player_stats_20162017 = pickle.load(pickle_in)

pickle_in = open("json_files/nhl_player_stats20172018.pickle","rb")
player_stats_20172018 = pickle.load(pickle_in)

pickle_in = open("json_files/nhl_player_stats20182019.pickle","rb")
player_stats_20182019 = pickle.load(pickle_in)

pickle_in = open("json_files/nhl_player_stats20192020.pickle","rb")
player_stats_20192020 = pickle.load(pickle_in)

pickle_in = open("json_files/nhl_player_stats20202021.pickle","rb")
player_stats_20202021 = pickle.load(pickle_in)

pickle_in = open("json_files/nhl_player_stats20212022.pickle","rb")
player_stats_20212022 = pickle.load(pickle_in)

#### Next we will convert our stored JSON files to dataframes

#### Note: Since we're using previous season stats to predict the next season salary we will not be import the players stats from the 2020-2021 season.

In [23]:
play_stats_20212022 = extract_player_stats(rosters, player_stats_20212022)
play_stats_20202021 = extract_player_stats(rosters, player_stats_20202021)
play_stats_20192020 = extract_player_stats(rosters, player_stats_20192020)
play_stats_20182019 = extract_player_stats(rosters, player_stats_20182019)
play_stats_20172018 = extract_player_stats(rosters, player_stats_20172018)
play_stats_20162017 = extract_player_stats(rosters, player_stats_20162017)
play_stats_20152016 = extract_player_stats(rosters, player_stats_20152016)
play_stats_20142015 = extract_player_stats(rosters, player_stats_20142015)
play_stats_20132014 = extract_player_stats(rosters, player_stats_20132014)

#### Next we add a suffix to our columns in each dataframe so that we can easily identify them after we join our dataframes.

In [24]:
play_stats_20212022 = play_stats_20212022.add_suffix('22')
play_stats_20202021 = play_stats_20202021.add_suffix('21')
play_stats_20192020 = play_stats_20192020.add_suffix('20')
play_stats_20182019 = play_stats_20182019.add_suffix('19')
play_stats_20172018 = play_stats_20172018.add_suffix('18')
play_stats_20162017 = play_stats_20162017.add_suffix('17')
play_stats_20152016 = play_stats_20152016.add_suffix('16')
play_stats_20142015 = play_stats_20142015.add_suffix('15')
play_stats_20132014 = play_stats_20132014.add_suffix('14')

#### Now we will rename the index column to link so that we have an identical column to join on.

In [25]:
play_stats_20212022 = play_stats_20212022.rename(columns={'index22': 'link'})
play_stats_20202021 = play_stats_20202021.rename(columns={'index21': 'link'})
play_stats_20192020 = play_stats_20192020.rename(columns={'index20': 'link'})
play_stats_20182019 = play_stats_20182019.rename(columns={'index19': 'link'})
play_stats_20172018 = play_stats_20172018.rename(columns={'index18': 'link'})
play_stats_20162017 = play_stats_20162017.rename(columns={'index17': 'link'})
play_stats_20152016 = play_stats_20152016.rename(columns={'index16': 'link'})
play_stats_20142015 = play_stats_20142015.rename(columns={'index15': 'link'})
play_stats_20132014 = play_stats_20132014.rename(columns={'index14': 'link'})

In [29]:
forward_roster = rosters[rosters['name'] != 'Goalie']
goalie_roster = rosters[rosters['name'] == 'Goalie']

In [30]:
forward_roster = forward_roster.join(play_stats_20212022.set_index('link'), on='link')
forward_roster = forward_roster.join(play_stats_20202021.set_index('link'), on='link')
forward_roster = forward_roster.join(play_stats_20192020.set_index('link'), on='link')
forward_roster = forward_roster.join(play_stats_20182019.set_index('link'), on='link')
forward_roster = forward_roster.join(play_stats_20172018.set_index('link'), on='link')
forward_roster = forward_roster.join(play_stats_20162017.set_index('link'), on='link')
forward_roster = forward_roster.join(play_stats_20152016.set_index('link'), on='link')
forward_roster = forward_roster.join(play_stats_20142015.set_index('link'), on='link')
forward_roster = forward_roster.join(play_stats_20132014.set_index('link'), on='link')

goalie_roster = goalie_roster.join(play_stats_20212022.set_index('link'), on='link')
goalie_roster = goalie_roster.join(play_stats_20202021.set_index('link'), on='link')
goalie_roster = goalie_roster.join(play_stats_20192020.set_index('link'), on='link')
goalie_roster = goalie_roster.join(play_stats_20182019.set_index('link'), on='link')
goalie_roster = goalie_roster.join(play_stats_20172018.set_index('link'), on='link')
goalie_roster = goalie_roster.join(play_stats_20162017.set_index('link'), on='link')
goalie_roster = goalie_roster.join(play_stats_20152016.set_index('link'), on='link')
goalie_roster = goalie_roster.join(play_stats_20142015.set_index('link'), on='link')
goalie_roster = goalie_roster.join(play_stats_20132014.set_index('link'), on='link')

#### Above we have a messy dataframe but we have all of the information that we need for now. Below we will start scraping our NHL player salaries.

# Web Scraping

#### Our second data source comes from [Spotrac.com](www.spotrac.com) through webscraping. This will populate all of our salary information for modeling.

#### We start by formatting our team names based on the URL specs needed for Spotrac. This will allow us to create a for loop when web scraping. 

In [31]:
all_teams_df_active = all_teams_df[all_teams_df['active'] == True]

team_names = []

for name in all_teams_df_active['name']:

    team_names.append(name.lower().replace(" ", "-"))
    
team_names = team_names[:31]

team_names

['new-jersey-devils',
 'new-york-islanders',
 'new-york-rangers',
 'philadelphia-flyers',
 'pittsburgh-penguins',
 'boston-bruins',
 'buffalo-sabres',
 'montréal-canadiens',
 'ottawa-senators',
 'toronto-maple-leafs',
 'carolina-hurricanes',
 'florida-panthers',
 'tampa-bay-lightning',
 'washington-capitals',
 'chicago-blackhawks',
 'detroit-red-wings',
 'nashville-predators',
 'st.-louis-blues',
 'calgary-flames',
 'colorado-avalanche',
 'edmonton-oilers',
 'vancouver-canucks',
 'anaheim-ducks',
 'dallas-stars',
 'los-angeles-kings',
 'san-jose-sharks',
 'columbus-blue-jackets',
 'minnesota-wild',
 'winnipeg-jets',
 'arizona-coyotes',
 'vegas-golden-knights']

#### The below cell scrapes spotrac by team name and stores all player links within the stats_link list. These URL's take us to a player specific webpage that contains the salaries for that player. These URL's will be used to attain all player salaries.

In [32]:
# stats_link = []

# for team in team_names:

#     url = f'https://www.spotrac.com/nhl/{team}/cap/2022'

#     spotrac = requests.get(url)

#     soup = BeautifulSoup(spotrac.text, 'html.parser')

#     base_url = 'https://www.spotrac.com/redirect'

#     for link in soup.find_all('a'):

#         url = link.get('href')

#         if url and '/player/' in url:

#             stats_link.append(url)

#### Below we get a quick look at the total number of players we will get information on which is 1384.

In [33]:
# len(stats_link)
print('2169')

2169


#### This next for loop eliminates URL's that bring us to a blank webpage. 

In [34]:
# stats_link = [item for item in stats_link if '/0/' not in item]

### By removing null URL's we have eliminated 200 total URLs to bring us to 1184 total players.

In [35]:
# len(stats_link)
print('2169')

2169


In [36]:
# name_dict = {}

# known_bad = []

# for i in stats_link:

#     url = i + 'cash-earnings/'
#     test_request = requests.get(url)
#     soup = BeautifulSoup(test_request.text, 'lxml')    


#     try:
#         name = soup.h1.text
#         name_dict[name] = {}
#         data_test = soup.find_all('tr', class_='salaryRow')[1:]
#         for years in data_test:
#             year = years.find('td', class_='salaryYear').text
#             salary = years.find('td', class_='salaryAmt cash').text
#             name_dict[name][year] = salary

#     except:

#         known_bad.append(i)

In [37]:
# pickle_out = open("json_files/nhl_player_salaries.pickle","wb")
# pickle.dump(name_dict, pickle_out)
# pickle_out.close()

In [38]:
pickle_in = open("json_files/nhl_player_salaries.pickle","rb")
nhl_player_salaries_base = pickle.load(pickle_in)

#### Moving forward, we will be joining our existing dataframe (from the NHL API) with our new salary dataframe. Since we do not have any matching columns besides player names we will use this column to join on. Below, we format the player names from Spotrac to match the player names from the NHL API.

In [39]:
money_norm = {}
for k,v in nhl_player_salaries_base.items():
    name = k.rstrip()
    money_norm[name] = {}
    for keys,vals in v.items():
        values = vals.strip('$').partition('(')
        money_norm[name][keys] = values[0]


#### Now we create our salary dataframe, rename our index column to match the fullName column in the NHL API, and format our data as integers. 

* Key Decision: All NaNs are replaced with 0. Later we will drop players if their salary in 2020-21 is 0 since this is the year that we will be predicting. 


In [40]:
nhl_salaries = pd.DataFrame(money_norm).T
nhl_salaries.head()

,2021-22,2020-21,2017-18,2018-19,2019-20,2013-14,2014-15,2015-16,2016-17,0-01,2010-11,2011-12,2012-13
Dougie Hamilton,"6,300,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Jack Hughes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Nico Hischier,"7,250,000","7,000,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Tomas Tatar,"3,750,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Damon Severson,"5,050,000","4,675,000","2,500,000","3,000,000","4,675,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
nhl_salaries = nhl_salaries.add_prefix('Salary_') \
                             .reset_index() \
                            .rename(columns={'index': 'fullName'}) \
                            .replace(',', '', regex=True) \
                            .replace('-', '0', regex=True) \
                            .set_index('fullName') \
                            .fillna('0') \
                            .astype('int64') \
                            .reset_index()
nhl_salaries.head()

,fullName,Salary_2021-22,Salary_2020-21,Salary_2017-18,Salary_2018-19,Salary_2019-20,Salary_2013-14,Salary_2014-15,Salary_2015-16,Salary_2016-17,Salary_0-01,Salary_2010-11,Salary_2011-12,Salary_2012-13
0,Dougie Hamilton,6300000,0,0,0,0,0,0,0,0,0,0,0,0
1,Jack Hughes,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Nico Hischier,7250000,7000000,0,0,0,0,0,0,0,0,0,0,0
3,Tomas Tatar,3750000,0,0,0,0,0,0,0,0,0,0,0,0
4,Damon Severson,5050000,4675000,2500000,3000000,4675000,0,0,0,0,0,0,0,0


In [42]:
nhl_salaries[nhl_salaries['fullName'] == 'Patrice Bergeron']

,fullName,Salary_2021-22,Salary_2020-21,Salary_2017-18,Salary_2018-19,Salary_2019-20,Salary_2013-14,Salary_2014-15,Salary_2015-16,Salary_2016-17,Salary_0-01,Salary_2010-11,Salary_2011-12,Salary_2012-13


In [47]:
forward_roster = forward_roster.set_index('fullName').join(nhl_salaries.set_index('fullName'), on='fullName').reset_index()

forward_roster.drop_duplicates(subset='fullName', inplace=True)

for col in forward_roster.columns[forward_roster.columns.str.contains('Salary')]:
    forward_roster[col] = forward_roster[col].fillna(0.0)

forward_roster.drop(['Salary_2013-14',
             'Salary_2011-12',
             'Salary_2012-13',
             'Salary_2010-11',
             'season22',
             'season21',
             'season20',
             'season19',
             'season18',
             'season17',
             'season16',
             'season15',
             'season14',
             'Salary_0-01'], axis=1, inplace=True)

goalie_roster = goalie_roster.set_index('fullName').join(nhl_salaries.set_index('fullName'), on='fullName').reset_index()

goalie_roster.drop_duplicates(subset='fullName', inplace=True)

for col in goalie_roster.columns[goalie_roster.columns.str.contains('Salary')]:
    goalie_roster[col] = goalie_roster[col].fillna(0.0)

goalie_roster.drop(['Salary_2013-14',
             'Salary_2011-12',
             'Salary_2012-13',
             'Salary_2010-11',
             'season22',
             'season21',
             'season20',
             'season19',
             'season18',
             'season17',
             'season16',
             'season15',
             'season14',
             'Salary_0-01'], axis=1, inplace=True)

In [ ]:
forward_roster.to_csv('data/forward_roster_df.csv')
goalie_roster.to_csv('data/goalie_roster_df.csv')